In [7]:
from typing import Literal, TypedDict

from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_deepseek import ChatDeepSeek
from langchain_core.messages import BaseMessage



# ========== 定义 State ==========
class AgentState(TypedDict):
    messages: list  # 存储对话消息
    step: int       # 循环步数计数


# ========== 定义 LLM ==========


llm = ChatDeepSeek(model="deepseek-chat", temperature=1.0)

# 工具（用 duckduckgo 搜索做演示）
search_tool = DuckDuckGoSearchRun()
tools = [search_tool]

# 绑定工具给 LLM
llm_with_tools = llm.bind_tools(tools)


# ========== 定义条件函数 ==========
MAX_STEPS = 5

def router(state: AgentState) -> Literal["tools", "continue", "end"]:
    """
    根据 LLM 最新输出，决定下一步走向：
    - 如果触发了 function_call → 调用工具
    - 如果达到最大步数 → 结束
    - 如果输出包含 FINAL_ANSWER → 结束
    - 否则继续思考
    """
    last_msg = state["messages"][-1]

    # 1. 如果触发工具调用
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "tools"

    # 2. 如果达到最大步数
    if state.get("step", 0) >= MAX_STEPS:
        return "end"

    # 3. 如果包含 FINAL_ANSWER 标记
    if "FINAL_ANSWER" in last_msg.content:
        return "end"

    # 4. 否则继续
    return "continue"


# ========== 构建 Graph ==========
workflow = StateGraph(AgentState)

# 节点：chatbot
def chatbot_node(state: AgentState):
    """LLM 节点，每次调用时步数 +1"""
    step = state.get("step", 0) + 1
    # 取出 messages（是 BaseMessage 列表）
    messages: list[BaseMessage] = state["messages"]

    # 调用模型
    response = llm_with_tools.invoke(messages)

    # 更新状态
    return {
        "messages": messages + [response],  # 把新消息加进去
        "step": step
    }

workflow.add_node("chatbot", chatbot_node)
workflow.add_node("tools", ToolNode(tools))

# chatbot 的条件边
workflow.add_conditional_edges("chatbot", router, {
    "tools": "tools",      # 调用工具
    "continue": "chatbot", # 自循环
    "end": END             # 结束
})

# 工具执行后回到 chatbot
workflow.add_edge("tools", "chatbot")

# 设置入口
workflow.set_entry_point("chatbot")

# 编译 graph
app = workflow.compile()


In [8]:
response = app.invoke({
    "messages": [("user", "帮我查一下LangGraph是做什么的，并给出结论。最终回答时请加上 FINAL_ANSWER")],
    "step": 0
})

print("循环次数:", response["step"])
print("最终回答:", response["messages"][-1].content)


BadRequestError: Error code: 400 - {'error': {'message': "Messages with role 'tool' must be a response to a preceding message with 'tool_calls'", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_request_error'}}